In [59]:
!pip install -q pyomo


In [60]:
from pyomo.environ import *

In [61]:
import numpy as np

$\ min\ x_1-2x_2+x_3 \\ \text{s.t.} \ x_1\geq1,\ x_2\geq2, x_3\geq1,\\ 2x_1 +x_2 +2x_3\leq10,\\ -x_1+x_2+2x_3\leq-3,\\ \ 2x_1-3x_2+x_3\leq-6$

In [62]:
model_ex1 = ConcreteModel()

In [63]:
N = 3
M = 3
lb = np.array([1,1,1])
ub = np.array([np.infty, np.infty, np.infty])
obj_coef_c = np.array([1,-2,3])
constr_coef_A = np.array([[2,1,2], [-1,1,2], [2,-3,1]])
constr_coef_rhs = np.array([10,-3,-6])
row_indices = np.arange(M)
print(row_indices)
col_indices = np.arange(N)
print(col_indices)

[0 1 2]
[0 1 2]


In [64]:
model_ex1.x = Var(col_indices)
model_ex1.constraint = ConstraintList()

In [65]:
for i in col_indices:
  model_ex1.x[i].setlb(lb[i])
  model_ex1.x[i].setub(ub[i])

In [66]:
for i in row_indices:
  model_ex1.constraint.add(sum(constr_coef_A[i][j]* model_ex1.x[j] for j in col_indices)<= constr_coef_rhs[i])

In [67]:
model_ex1.objective = Objective(expr = summation(obj_coef_c, model_ex1.x), sense= minimize)

In [68]:
model_ex1.pprint()

2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     1 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraint : Size=3, Index=constraint_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10.0 :  

In [69]:
!apt-get install -y -qq glpk-utils

In [70]:
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
result = opt.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)



Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01413869857788086

Solver status: ok
Solver termination condition: other


In [71]:
!apt-get install -y -qq coinor-cbc

In [72]:
opt_cbc = SolverFactory('cbc')

In [73]:
result = opt_cbc.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver termination message: ', result.solver.termination_message)

print('We are getting infeasible solution. This may be due to some constraints to which solution isnot satisfying.')

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.016495466232299805

Solver status: warning
Solver termination condition: infeasible
Solver termination message:  Model was proven to be infeasible.
We are getting infeasible solution. This may be due to some constraints to which solution isnot satisfying.


**a.**

In [74]:
model_ex1.objective.set_sense(maximize)
model_ex1.pprint()

2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     1 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraint : Size=3, Index=constraint_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10.0 :  

In [75]:
result = opt_cbc.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver termination message: ', result.solver.termination_message)

print('We are getting infeasible solution. This may be due to some constraints to which solution isnot satisfying.')

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: None
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.022550106048583984

Solver status: warning
Solver termination condition: infeasible
Solver termination message:  Model was proven to be infeasible.
We are getting infeasible solution. This may be due to some constraints to which solution isnot satisfying.


In [76]:
model_ex1.objective.set_sense(minimize)

**b.**

In [77]:
model_ex1.x[1].setub(8)
model_ex1.pprint()

2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     1 :  None :     8 : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraint : Size=3, Index=constraint_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10.0 :  

In [78]:
result = opt_cbc.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver termination message: ', result.solver.termination_message)

print('We are getting infeasible solution. This may be due to some constraints to which solution isnot satisfying.')

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.017477989196777344

Solver status: warning
Solver termination condition: infeasible
Solver termination message:  Model was proven to be infeasible.
We are getting infeasible solution. This may be due to some constraints to which solution isnot satisfying.


In [79]:
model_ex1.x[1].setub(np.infty)

**c.**

In [80]:
new_constr_coef = [0,-1,1]
new_constr_rhs = 9
model_ex1.constraint.add(sum(new_constr_coef[j]*model_ex1.x[j] for j in col_indices) <= new_constr_rhs)
model_ex1.pprint()

2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     1 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraint : Size=4, Index=constraint_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10.0 

In [81]:
result = opt_cbc.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver termination message: ', result.solver.termination_message)

print('We are getting infeasible solution. This may be due to some constraints to which solution isnot satisfying.')

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.02237105369567871

Solver status: warning
Solver termination condition: infeasible
Solver termination message:  Model was proven to be infeasible.
We are getting infeasible solution. This may be due to some constraints to which solution isnot satisfying.


In [82]:
model_ex1.constraint[4].deactivate()

**d.**

In [83]:
model_ex1.objective.deactivate()
model_ex1.constraint[1].deactivate()
new2_constr_coef = [-1,-1,0]
new2_constr_rhs = -45
model_ex1.constraint.add(sum(new2_constr_coef[j]*model_ex1.x[j] for j in col_indices) <= new2_constr_rhs)
new_obj_coef = np.array([1,1,1])
model_ex1.new_objective = Objective(expr =summation(new_obj_coef,model_ex1.x))
model_ex1.pprint()

2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     1 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] + x[1] + x[2]
    objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraint : Siz

In [84]:
result = opt_cbc.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver termination message: ', result.solver.termination_message)


print('We got optimal solution, because all the constraints have been satisfied.')


Problem: 
- Name: unknown
  Lower bound: 46.0
  Upper bound: 46.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 2
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.019381284713745117
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver termination message:  Model was solved to optimality (subject to tolerances), and an optimal solution is available.
We got optimal solution, because all the constraints have been satisfied.


In [85]:
# display solution
print('\nObjective = ', model_ex1.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_ex1.x[i].value)
print('\nConstraints')
model_ex1.constraint.display()
print('\n       5th constraint has satisfied equality condition.')


Objective =  46.0

Decision Variables
x[ 0 ] =  25.6
x[ 1 ] =  19.4
x[ 2 ] =  1.0

Constraints
constraint : Size=5
    Key : Lower : Body               : Upper
      2 :  None : -4.200000000000003 :  -3.0
      3 :  None : -5.999999999999993 :  -6.0
      5 :  None :              -45.0 : -45.0

       5th constraint has satisfied equality condition.
